In order to run this notebook with the correct PYTHONPATH, you can use
```
$ ./e-mission-jupyter.bash notebook
```

This notebook makes heavy use of the following python libraries.
- `pandas`: from the python computing stack (http://pandas.pydata.org/pandas-docs/stable/)
- `geojson`: standard JSON representation of geographic data (http://geojson.org/)
- `folium`: (https://github.com/python-visualization/folium) python bridge to leaflet (http://leafletjs.com/)

### Pick a user to work with ###

In [1]:
import emission.core.get_database as edb
import pandas as pd

all_users = pd.DataFrame(list(edb.get_uuid_db().find({}, {"user_email":1, "uuid": 1, "_id": 0})))
all_users

storage not configured, falling back to sample, default configuration
Connecting to database URL localhost


,user_email,uuid
0,emma,77dad2b8-ab85-40d1-a741-2c5d7c578ac5
1,bill,b7e58f40-8e5d-4a21-82ac-5a90dce44647
2,sam,a011258d-fb21-4e23-bc05-03777ea8ea72
3,shankari,7a0359dc-a208-4224-9a1e-441224860927
4,test_user_1,9a9b2074-ed61-468a-935c-8362adc57873
5,car_user,ddfcca5c-d5c9-43de-8056-c3a2aa27d0a7
6,samanthatrip,c07b0f77-e591-44a8-a9c0-ac30babf3c63
7,billtrip,d7f43d82-7adb-43d8-a7fb-946c9dce5d6b
8,trevortrip,c7cbe4a6-a5fb-4b02-8870-0c78aeeadefd
9,samtripfail,fc201e64-5e9d-474e-bc3c-0ab91058939b


In [2]:



from uuid import UUID

In [3]:
test_user_id = all_users.iloc[9].uuid # replace with UUID from above
print(test_user_id)

fc201e64-5e9d-474e-bc3c-0ab91058939b




If you want to work across multiple users, just do the same thing again

In [4]:
test_user_id_2 = all_users.iloc[2].uuid


### Preferred access technique




The preferred technique to access wrapper objects from the timeseries is to use the abstract timeseries interface. This makes it easier for us to switch to alternative timeseries implementations later. The timeseries is conceptually a set of streams, one for each of the types, primarily indexed by time. So you can query for all entries of a particular time within a specified time range.

In [5]:


import emission.storage.timeseries.abstract_timeseries as esta
import emission.storage.decorations.analysis_timeseries_queries as esda
import emission.core.wrapper.entry as ecwe
import emission.storage.decorations.trip_queries as esdt
import emission.storage.timeseries.timequery as estt
import emission.core.wrapper.suggestion_sys as sug
import json

In [6]:




print(test_user_id, test_user_id_2)
ts = esta.TimeSeries.get_time_series(test_user_id)
ts_2 = esta.TimeSeries.get_time_series(test_user_id_2)
#CLEANED_TRIP_KEY 


fc201e64-5e9d-474e-bc3c-0ab91058939b a011258d-fb21-4e23-bc05-03777ea8ea72


In [7]:
sug.calculate_yelp_server_suggestion_nominatim(test_user_id)

['desserts', 'icecream', 'catering']
{'hasTollRoad': False, 'hasBridge': True, 'boundingBox': {'lr': {'lng': -122.188164, 'lat': 37.711544}, 'ul': {'lng': -122.276199, 'lat': 37.870583}}, 'distance': 14.647, 'hasTimedRestriction': False, 'hasTunnel': False, 'hasHighway': True, 'computedWaypoints': [], 'routeError': {'errorCode': -400, 'message': ''}, 'formattedTime': '00:22:11', 'sessionId': '5c46a538-0176-5f21-02b4-1e65-026d88872334', 'hasAccessRestriction': False, 'realTime': 10000000, 'hasSeasonalClosure': False, 'hasCountryCross': False, 'fuelUsed': 0.74, 'legs': [{'hasTollRoad': False, 'hasBridge': True, 'destNarrative': 'Proceed to 2132 OXFORD LN.', 'distance': 14.647, 'hasTimedRestriction': False, 'hasTunnel': False, 'hasHighway': True, 'index': 0, 'formattedTime': '00:22:11', 'origIndex': 3, 'hasAccessRestriction': False, 'hasSeasonalClosure': False, 'hasCountryCross': False, 'roadGradeStrategy': [[]], 'destIndex': 8, 'time': 1331, 'hasUnpaved': False, 'origNarrative': 'Go nort

{'hasTollRoad': False, 'hasBridge': True, 'boundingBox': {'lr': {'lng': -122.188164, 'lat': 37.711544}, 'ul': {'lng': -122.276199, 'lat': 37.86676}}, 'distance': 14.904, 'hasTimedRestriction': False, 'hasTunnel': False, 'hasHighway': True, 'computedWaypoints': [], 'routeError': {'errorCode': -400, 'message': ''}, 'formattedTime': '00:20:22', 'sessionId': '5c46a538-038b-5f21-02b4-1f0c-06cfabc64ce4', 'hasAccessRestriction': False, 'realTime': 10000000, 'hasSeasonalClosure': False, 'hasCountryCross': False, 'fuelUsed': 0.73, 'legs': [{'hasTollRoad': False, 'hasBridge': True, 'destNarrative': 'Proceed to BERKELEY, CA 94704.', 'distance': 14.904, 'hasTimedRestriction': False, 'hasTunnel': False, 'hasHighway': True, 'index': 0, 'formattedTime': '00:20:22', 'origIndex': 3, 'hasAccessRestriction': False, 'hasSeasonalClosure': False, 'hasCountryCross': False, 'roadGradeStrategy': [[]], 'destIndex': 6, 'time': 1222, 'hasUnpaved': False, 'origNarrative': 'Go north on I-880 N.', 'maneuvers': [{'di

{'hasTollRoad': False, 'hasBridge': True, 'boundingBox': {'lr': {'lng': -121.320518, 'lat': 37.69046}, 'ul': {'lng': -122.211136, 'lat': 37.973648}}, 'distance': 68.413, 'hasTimedRestriction': False, 'hasTunnel': False, 'hasHighway': True, 'computedWaypoints': [], 'routeError': {'errorCode': -400, 'message': ''}, 'formattedTime': '01:07:06', 'sessionId': '5c46a539-01d4-5f21-02b4-1f17-0a9b3ea58d2a', 'hasAccessRestriction': False, 'realTime': 4204, 'hasSeasonalClosure': False, 'hasCountryCross': False, 'fuelUsed': 3.18, 'legs': [{'hasTollRoad': False, 'hasBridge': True, 'destNarrative': 'Proceed to 2575 W TELEGRAPH AVE.', 'distance': 68.413, 'hasTimedRestriction': False, 'hasTunnel': False, 'hasHighway': True, 'index': 0, 'formattedTime': '01:07:06', 'origIndex': 3, 'hasAccessRestriction': False, 'hasSeasonalClosure': False, 'hasCountryCross': False, 'roadGradeStrategy': [[]], 'destIndex': 9, 'time': 4026, 'hasUnpaved': False, 'origNarrative': 'Go south on I-880 S.', 'maneuvers': [{'dist

{'hasTollRoad': False, 'hasBridge': True, 'boundingBox': {'lr': {'lng': -119.26001, 'lat': 36.323147}, 'ul': {'lng': -122.211136, 'lat': 37.784863}}, 'distance': 213.23, 'hasTimedRestriction': False, 'hasTunnel': False, 'hasHighway': True, 'computedWaypoints': [], 'routeError': {'errorCode': -400, 'message': ''}, 'formattedTime': '03:19:08', 'sessionId': '5c46a53a-0019-5f21-02b4-1f0c-06cfabc64ce4', 'hasAccessRestriction': False, 'realTime': 12817, 'hasSeasonalClosure': False, 'hasCountryCross': False, 'fuelUsed': 9.99, 'legs': [{'hasTollRoad': False, 'hasBridge': True, 'destNarrative': 'Proceed to 2948 E COLLEGE AVE.', 'distance': 213.23, 'hasTimedRestriction': False, 'hasTunnel': False, 'hasHighway': True, 'index': 0, 'formattedTime': '03:19:08', 'origIndex': 3, 'hasAccessRestriction': False, 'hasSeasonalClosure': False, 'hasCountryCross': False, 'roadGradeStrategy': [[]], 'destIndex': 12, 'time': 11948, 'hasUnpaved': False, 'origNarrative': 'Go south on I-880 S.', 'maneuvers': [{'dis

{'hasTollRoad': False, 'hasBridge': True, 'boundingBox': {'lr': {'lng': -117.60865, 'lat': 34.091076}, 'ul': {'lng': -122.211136, 'lat': 37.731071}}, 'distance': 400.26, 'hasTimedRestriction': False, 'hasTunnel': True, 'hasHighway': True, 'computedWaypoints': [], 'routeError': {'errorCode': -400, 'message': ''}, 'formattedTime': '06:02:09', 'sessionId': '5c46a53a-02bc-5f21-02b4-1e65-026d88872334', 'hasAccessRestriction': False, 'realTime': 23654, 'hasSeasonalClosure': False, 'hasCountryCross': False, 'fuelUsed': 19.22, 'legs': [{'hasTollRoad': False, 'hasBridge': True, 'destNarrative': 'Proceed to 1819 N SOLANO AVE.', 'distance': 400.26, 'hasTimedRestriction': False, 'hasTunnel': True, 'hasHighway': True, 'index': 0, 'formattedTime': '06:02:09', 'origIndex': 3, 'hasAccessRestriction': False, 'hasSeasonalClosure': False, 'hasCountryCross': False, 'roadGradeStrategy': [[]], 'destIndex': 14, 'time': 21729, 'hasUnpaved': False, 'origNarrative': 'Go south on I-880 S.', 'maneuvers': [{'dista

...

{'hasTollRoad': False, 'hasBridge': True, 'boundingBox': {'lr': {'lng': -118.250519, 'lat': 33.928493}, 'ul': {'lng': -118.401398, 'lat': 34.07951}}, 'distance': 20.527, 'hasTimedRestriction': False, 'hasTunnel': True, 'hasHighway': True, 'computedWaypoints': [], 'routeError': {'errorCode': -400, 'message': ''}, 'formattedTime': '00:24:19', 'sessionId': '5c46a551-01c2-5f21-02b4-1f17-0a9b3ea58d2a', 'hasAccessRestriction': False, 'realTime': 1588, 'hasSeasonalClosure': False, 'hasCountryCross': False, 'fuelUsed': 0.96, 'legs': [{'hasTollRoad': False, 'hasBridge': True, 'destNarrative': 'Proceed to [1200 - 3899] W SUNSET BLVD, AND SUTHERLAND STLOS.', 'distance': 20.527, 'hasTimedRestriction': False, 'hasTunnel': True, 'hasHighway': True, 'index': 0, 'formattedTime': '00:24:19', 'origIndex': 4, 'hasAccessRestriction': False, 'hasSeasonalClosure': False, 'hasCountryCross': False, 'roadGradeStrategy': [[]], 'destIndex': 10, 'time': 1459, 'hasUnpaved': False, 'origNarrative': 'Go south on Pac

['burgers', 'hotdogs']
{'hasTollRoad': False, 'hasBridge': True, 'boundingBox': {'lr': {'lng': -118.227676, 'lat': 33.891838}, 'ul': {'lng': -118.406242, 'lat': 34.123497}}, 'distance': 25.751, 'hasTimedRestriction': False, 'hasTunnel': True, 'hasHighway': True, 'computedWaypoints': [], 'routeError': {'errorCode': -400, 'message': ''}, 'formattedTime': '00:30:26', 'sessionId': '5c46a557-0248-5f21-02b4-1f0c-06cfabc64ce4', 'hasAccessRestriction': False, 'realTime': 1956, 'hasSeasonalClosure': False, 'hasCountryCross': False, 'fuelUsed': 1.18, 'legs': [{'hasTollRoad': False, 'hasBridge': True, 'destNarrative': 'Proceed to 3179 LOS FELIZ BLVD.', 'distance': 25.751, 'hasTimedRestriction': False, 'hasTunnel': True, 'hasHighway': True, 'index': 0, 'formattedTime': '00:30:26', 'origIndex': 3, 'hasAccessRestriction': False, 'hasSeasonalClosure': False, 'hasCountryCross': False, 'roadGradeStrategy': [[]], 'destIndex': 10, 'time': 1826, 'hasUnpaved': False, 'origNarrative': 'Go north on Pacific C

{'hasTollRoad': False, 'hasBridge': True, 'boundingBox': {'lr': {'lng': -118.250519, 'lat': 33.891838}, 'ul': {'lng': -118.406242, 'lat': 34.07951}}, 'distance': 22.154, 'hasTimedRestriction': False, 'hasTunnel': False, 'hasHighway': True, 'computedWaypoints': [], 'routeError': {'errorCode': -400, 'message': ''}, 'formattedTime': '00:27:42', 'sessionId': '5c46a558-011e-5f21-02b4-1f0c-06cfabc64ce4', 'hasAccessRestriction': False, 'realTime': 1809, 'hasSeasonalClosure': False, 'hasCountryCross': False, 'fuelUsed': 1.03, 'legs': [{'hasTollRoad': False, 'hasBridge': True, 'destNarrative': 'Proceed to [1200 - 3899] W SUNSET BLVD, AND SUTHERLAND STLOS.', 'distance': 22.154, 'hasTimedRestriction': False, 'hasTunnel': False, 'hasHighway': True, 'index': 0, 'formattedTime': '00:27:42', 'origIndex': 3, 'hasAccessRestriction': False, 'hasSeasonalClosure': False, 'hasCountryCross': False, 'roadGradeStrategy': [[]], 'destIndex': 11, 'time': 1662, 'hasUnpaved': False, 'origNarrative': 'Go north on P

['newamerican', 'tapasmallplates', 'cocktailbars']


{'end_lat': '0.0',
 'end_lon': '0.0',
 'message': 'Good job walking and biking! No suggestion to show.',
 'method': 'bike',
 'savings': '0',
 'start_lat': '0.0',
 'start_lon': '0.0'}


#### Accessing entries directly

In [ ]:

# Get all cleaned trips for the first user
entry_it = ts.find_entries(["analysis/cleaned_trip"], time_query=None)



All keys and their mapping to data model objects can be found in 
https://github.com/e-mission/e-mission-server/blob/master/emission/core/wrapper/entry.py

In [ ]:
print(sug.calculate_yelp_server_suggestion_nominatim(test_user_id))
for ct in entry_it:
    cte = ecwe.Entry(ct)
    print("=== Trip:", cte.data.start_loc, "->", cte.data.end_loc)
    section_it = esdt.get_sections_for_trip("analysis/cleaned_section", test_user_id, cte.get_id())
    for sec in section_it:
        print("  --- Section:", sec.data.start_loc, "->", sec.data.end_loc, " on ", sec.data.sensed_mode)

In [ ]:


# Get all cleaned trips for the second user
entry_it = ts_2.find_entries(["analysis/cleaned_trip"], time_query=None)

In [ ]:
for ct in entry_it:
    cte = ecwe.Entry(ct)
    print("=== Trip:", cte.data.start_loc, "->", cte.data.end_loc)
    section_it = esdt.get_sections_for_trip("analysis/cleaned_section", test_user_id, cte.get_id())
    for sec in section_it:
        print("  --- Section:", sec.data.start_loc, "->", sec.data.end_loc, " on ", sec.data.sensed_mode)

In [ ]:
# Get cleaned trips for the two users that started on 1st Aug UTC
import arrow

aug_1_tq = estt.TimeQuery("data.start_ts",
                          arrow.get("2017-08-01").timestamp, # start of range
                          arrow.get("2017-08-02").timestamp)  # end of range
entry_it = ts.find_entries(["analysis/cleaned_trip"], time_query=aug_1_tq)
entry_it_2 = ts_2.find_entries(["analysis/cleaned_trip"], time_query=aug_1_tq)
print("From %s -> %s, user %s had %d trips and user %s had %d trips" %
     (aug_1_tq.startTs, aug_1_tq.endTs, test_user_id, len(list(entry_it)), test_user_id_2, len(list(entry_it_2))))


#### Accessing a dataframe

In [ ]:
# Get all cleaned trips for the first user
ct_df = ts.get_data_df("analysis/cleaned_trip", time_query=None)

In [ ]:
len(ct_df)





In [ ]:
ct_df.columns

In [ ]:
# print(ct_df[["_id","start_loc", "end_loc", "start_fmt_time", "end_fmt_time"]])
test_id = ct_df.iloc[0]._id
# user_id = all_users.iloc[all_users[all_users.uuid == uuid].index.tolist()[0]].uuid
cleaned = ts.get_entry_from_id("analysis/cleaned_trip", test_id)
car = cleaned.get("data")
print(cleaned.data.distance)

# spec_trip = ct_df.iloc[ct_df[ct_df._id == test_id].index.tolist()[0]]

print(sug.calculate_yelp_server_suggestion_singletrip_nominatim(test_user_id, test_id))
# ct_df.iloc[ct_df[ct_df._id == '5bc6a2f343de75fa8e9b2568']]



In [ ]:
# Get all cleaned trips for the second user
ct_df_2 = ts_2.get_data_df("analysis/cleaned_trip", time_query=None)
ct_df_2[["start_loc", "end_loc", "start_ts", "end_ts"]]

In [ ]:
# Get cleaned trips for the two users that started on 1st Aug UTC
import arrow

aug_1_tq = estt.TimeQuery("data.start_ts",
                          arrow.get("2017-08-01").timestamp, # start of range
                          arrow.get("2017-08-02").timestamp)  # end of range
ct_df = ts.get_data_df("analysis/cleaned_trip", time_query=aug_1_tq)
ct_df_2 = ts_2.get_data_df("analysis/cleaned_trip", time_query=aug_1_tq)
print("From %s -> %s, user %s had %d trips and user %s had %d trips" %
     (aug_1_tq.startTs, aug_1_tq.endTs, test_user_id, len(ct_df), test_user_id_2, len(ct_df_2)))

In [ ]:
cs_df = ts.get_data_df("analysis/cleaned_section", time_query=None)

In [ ]:
len(cs_df)

In [ ]:
cs_df.columns

In [ ]:
cs_df[["start_loc", "end_loc", "start_ts", "end_ts", "sensed_mode"]]

### Direct mongodb queries

You can also use direct mongodb queries during exploratory work. I do ask that you create a storage decorator (`emission/storage/decorations`) when you submit a pull request for ongoing use

In [ ]:
import emission.core.get_database as edb

In [ ]:
edb.get_timeseries_db().find_one()

In [ ]:
edb.get_timeseries_db().distinct("metadata.key")

Note that in this case, you need to know whether to use the `timeseries` or the `analysis_timeseries` collection

In [ ]:
edb.get_analysis_timeseries_db().distinct("metadata.key")

In [ ]:
edb.get_analysis_timeseries_db().find({"user_id": test_user_id, "metadata.key": "analysis/cleaned_trip"}).count()

In particular, you can use this to access entries that are not in the timeseries

In [ ]:
edb.get_uuid_db().distinct("uuid")

### Timeline

The trips and places maintain links to each other - e.g. `start_place`, `end_place`

In [ ]:
ct_df[["start_place", "end_place"]]

These are _primary key links_ to other entries in the database. It would be useful to have a doubly linked list representing this properly. The Timeline helps with that.

In [ ]:
import emission.storage.decorations.timeline as esdl

In [ ]:
trip_start_end_fuzz = 10 # seconds
ct_df = ts.get_data_df("analysis/cleaned_trip", time_query=None)
tl = esdl.get_cleaned_timeline(test_user_id, ct_df.iloc[0].start_ts - trip_start_end_fuzz, ct_df.iloc[-1].end_ts + trip_start_end_fuzz)

In [ ]:
for e in tl:
    if 'enter_ts' in e.data:
        # Must be place-like
        print(e.metadata.key, e.data.enter_fmt_time, "->", e.data.exit_fmt_time)
    else:
        print(e.metadata.key, e.data.start_fmt_time, "->", e.data.end_fmt_time)

In [ ]:
# The timeline is an iterator, so after it is consumed, it is empty
for e in tl:
    if 'enter_ts' in e.data:
        # Must be place-like
        print(e.metadata.key, e.data.enter_fmt_time, "->", e.data.exit_fmt_time)
    else:
        print(e.metadata.key, e.data.start_fmt_time, "->", e.data.end_fmt_time)

In [ ]:
stl = esdt.get_cleaned_timeline_for_trip(test_user_id, tl.first_trip().get_id())

In [ ]:
for e in stl:
    print(e.metadata.key)

In [ ]:
stl = esdt.get_cleaned_timeline_for_trip(test_user_id, tl.last_trip().get_id())

In [ ]:
for e in stl:
    print(e.metadata.key)

## Getting trip and section details ##

Once we have trip and section objects, we can retrieve the sensed data associated with them by querying for data in various streams that falls within the time ranges associated with the trip/section. Here again, our architecture of storing the analysis results as a separate datastream makes it easy to retrieve data at various levels of processing.

### Plot a processed trip or set of trips ###

In [ ]:
import emission.analysis.plotting.geojson.geojson_feature_converter as gfc
import emission.analysis.plotting.leaflet_osm.our_plotter as lo

In [ ]:
first_trip_for_user = ct_df.iloc[0]
first_trip_start_ts = first_trip_for_user.start_ts
first_trip_end_ts = first_trip_for_user.end_ts
trip_start_end_fuzz = 10 # seconds
trips_geojson_list = gfc.get_geojson_for_ts(test_user_id, first_trip_start_ts-trip_start_end_fuzz, ct_df.iloc[-1].end_ts+trip_start_end_fuzz)

In [ ]:
len(trips_geojson_list)

In [ ]:
map_list = lo.get_maps_for_geojson_trip_list(trips_geojson_list)

In [ ]:
len(map_list)

In [ ]:
map_list[0]

In [ ]:
map_list[-1]

In [ ]:
import branca.element as bre

In [ ]:
nrows = 2
ncols = 3
fig = bre.Figure()
for i, m in enumerate(map_list[:6]):
    fig.add_subplot(nrows,ncols,i+1).add_child(m)
fig

In [ ]:
nrows = 2
ncols = 3
fig = bre.Figure()
for i, map in enumerate(map_list[-6:]):
    fig.add_subplot(nrows,ncols,i+1).add_child(map)
fig

## Can you do better? ##


### Get locations with no processing, basic filtering and resampling for the first trip ###

In [ ]:
all_locs = ts.get_data_df("background/location",
                               time_query = esda.get_time_query_for_trip_like(
                                   "analysis/cleaned_section", ct_df.iloc[0]._id))

filtered_locs = ts.get_data_df("background/filtered_location",
                               time_query = esda.get_time_query_for_trip_like(
                                   "analysis/cleaned_section", ct_df.iloc[0]._id))

resampled_locs = ts.get_data_df("analysis/recreated_location",
                                 time_query = esda.get_time_query_for_trip_like(
                                   "analysis/cleaned_section", ct_df.iloc[0]._id))
print("Locations go from all = %d -> filtered = %d -> resampled = %d" % (len(all_locs),
                                                                         len(filtered_locs),
                                                                         len(resampled_locs)))

In [ ]:
all_locs[["_id", "latitude", "longitude", "fmt_time"]]

### Get the raw motion activity, in case you want to do different segmentation ###

In [ ]:
all_activity = ts.get_data_df("background/motion_activity",
                               esda.get_time_query_for_trip_like(
                                   "analysis/cleaned_section", ct_df.iloc[0]._id))

In [ ]:
all_activity.columns

In [ ]:
import emission.core.wrapper.motionactivity as ecwm

print("Found %d walking entries, %d on_foot entries" % (len(all_activity[all_activity.type == ecwm.MotionTypes.WALKING.value]), 
                                    len(all_activity[all_activity.type == ecwm.MotionTypes.ON_FOOT.value])))
print("Found %d motorized entries" % (len(all_activity[all_activity.type == ecwm.MotionTypes.IN_VEHICLE.value])))

### Plot the location points ###

In [ ]:
map_list = lo.get_maps_for_geojson_unsectioned([gfc.get_feature_list_from_df(all_locs),
                                                gfc.get_feature_list_from_df(filtered_locs),
                                                gfc.get_feature_list_from_df(resampled_locs)])

In [ ]:
fig = bre.Figure()
for i, map in enumerate(map_list):
    fig.add_subplot(1,3,i+1).add_child(map)
fig